# This script gets all the data from an online database and inserts it into postgresql/postgis.

## Loading the database and all the pertinent libraries

In [ ]:
try:
    import psycopg2
    import pandas as pd
    import pandas.io.sql as psql
    import numpy as np
    from matplotlib import pyplot as plt
    from IPython.display import display
    import os
    import pandas_profiling as pp
    import requests 
    from shapely.geometry import Point,Polygon,MultiPolygon, mapping 
    import datetime




    database = "practical_sql"
    user = "postgres"
    password = 3715
    host = "localhost"
    port = "5432"

    conn = psycopg2.connect(database = database, user = user, password =password, host = host, port = port)

    print ("Opened database successfully")

except:
    print("------something went wrong------")

### Here we can make sure that the all rows and columns are displayed.

In [10]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [26]:
"""cursor = conn.cursor()
cursor.execute("CREATE TABLE areacommands (id SERIAL PRIMARY KEY, name VARCHAR(20), geom GEOMETRY)")
cursor.execute("CREATE TABLE beats (id SERIAL PRIMARY KEY, beat VARCHAR(6), agency VARCHAR(3), areacomm VARCHAR(15),geom GEOMETRY)")
cursor.execute("CREATE TABLE incidents (id SERIAL PRIMARY KEY, address VARCHAR(72), crimetype VARCHAR(255), date DATE,geom GEOMETRY)")
conn.commit() """

print("Data has been successfuly created!!")

Data has been successfuly created!!


In [24]:
url='http://coagisweb.cabq.gov/arcgis/rest/services/public/adminboundaries/MapServer/8/query' 
params={"where":"1=1","outFields":"*","outSR":"4326","f":"json"} 
r=requests.get(url,params=params) 
data=r.json()
for acmd in data['features']:    
    polys=[]
    for ring in acmd['geometry']['rings']:        
        polys.append(Polygon(ring))    
    p=MultiPolygon(polys)    
    name=acmd['attributes']['Area_Command']    
    cursor.execute("INSERT INTO areacommands (name, geom) VALUES ('{}', ST_GeomFromText('{}'))".format(name, p.wkt))
conn.commit() 
 

In [42]:
query = """

select * 
from areacommands



"""

data = psql.read_sql(query, conn)
data

,id,name,geom
0,1,FOOTHILLS,0106000000050000000103000000010000006A050000D9...
1,2,NORTHEAST,010600000003000000010300000001000000F1020000C3...
2,3,NORTHWEST,010600000006000000010300000001000000D6110000C1...
3,4,SOUTHEAST,0106000000030000000103000000010000006D030000BF...
4,5,SOUTHWEST,0106000000160000000103000000010000000E1000008F...
5,6,VALLEY,010600000012000000010300000001000000820D000085...
6,7,FOOTHILLS,0106000000050000000103000000010000006A050000D9...
7,8,NORTHEAST,010600000003000000010300000001000000F1020000C3...
8,9,NORTHWEST,010600000006000000010300000001000000D6110000C1...
9,10,SOUTHEAST,0106000000030000000103000000010000006D030000BF...


In [41]:
url='http://coagisweb.cabq.gov/arcgis/rest/services/public/adminboundaries/MapServer/9/query' 
params={"where":"1=1","outFields":"*","outSR":"4326","f":"json"}
r=requests.get(url,params=params) 
data=r.json()
for acmd in data['features']:    
    polys=[]    
    for ring in acmd['geometry']['rings']:        
        polys.append(Polygon(ring))    
    p=MultiPolygon(polys)
    beat = acmd['attributes']['BEAT']    
    agency = acmd['attributes']['AGENCY']    
    areacomm = acmd['attributes']['AREA_COMMA']
    cursor.execute("INSERT INTO beats (beat, agency,areacomm,geom) VALUES ('{}','{}','{}', ST_GeomFromText('{}'))".format(beat,agency,areacomm,p.wkt))
conn.commit() 
